In [1]:
import pandas as pd
import numpy as np

df = pd.read_excel('electronika_clean.xlsx')

In [2]:
# Выручка / количество товаров

fs1 = df[['Phone_new', 'Order_ID', 'Quant', 'FullSum']] \
        .groupby(['Phone_new']) \
        .aggregate({'Order_ID': 'nunique', 'Quant': 'sum', 'FullSum': 'sum'}) \
        .rename(columns={'Order_ID': 'Чеки', 'Quant': 'Товары', 'FullSum': 'Выручка'})
fs1['Товар/чек'] = (fs1['Товары'] / fs1['Чеки']).apply(lambda x: round(x, 2))
fs1['Средний_чек'] = (fs1['Выручка'] / fs1['Чеки']).apply(lambda x: round(x, 2))

fs2 = df[df['Status_ID'] == 14][['Phone_new', 'Order_ID', 'Quant', 'FullSum']] \
        .groupby(['Phone_new']) \
        .aggregate({'Order_ID': 'nunique', 'Quant': 'sum', 'FullSum': 'sum'}) \
        .rename(columns={'Order_ID': 'Чеки_выкуп', 'Quant': 'Товары_выкуп', 'FullSum': 'Выручка_выкуп'})
fs2['Товар/чек_выкуп'] = (fs2['Товары_выкуп'] / fs2['Чеки_выкуп']).apply(lambda x: round(x, 2))
fs2['Средний_чек_выкуп'] = (fs2['Выручка_выкуп'] / fs2['Чеки_выкуп']).apply(lambda x: round(x, 2))


fs3 = fs1.join(fs2, how='left')
fs3.fillna(0, inplace=True)
fs3['процент_выкупа'] = fs3['Выручка_выкуп'] / fs3['Выручка']
fs3

,Чеки,Товары,Выручка,Товар/чек,Средний_чек,Чеки_выкуп,Товары_выкуп,Выручка_выкуп,Товар/чек_выкуп,Средний_чек_выкуп,процент_выкупа
Phone_new,,,,,,,,,,,
32565748-000000000,13,15,259458.0,1.15,19958.31,8.0,8.0,19757.0,1.0,2469.62,0.076147
32565748-535549565054 55,1,1,12430.0,1.00,12430.00,0.0,0.0,0.0,0.0,0.00,0.000000
32565750-575448565151 а,1,2,53198.0,2.00,53198.00,1.0,2.0,53198.0,2.0,53198.00,1.000000
32565750-575448565151 а5,1,1,1590.0,1.00,1590.00,1.0,1.0,1590.0,1.0,1590.00,1.000000
43553240-575054413251+75,7,7,48557.0,1.00,6936.71,2.0,2.0,10129.0,1.0,5064.50,0.208600
...,...,...,...,...,...,...,...,...,...,...,...
57505548-48535350504991,1,1,999.0,1.00,999.00,1.0,1.0,999.0,1.0,999.00,1.000000
57544955-51485756555597,1,1,7700.0,1.00,7700.00,0.0,0.0,0.0,0.0,0.00,0.000000
57555656-55575253555698,1,1,4040.0,1.00,4040.00,1.0,1.0,4040.0,1.0,4040.00,1.000000


In [3]:
# Остальные признаки

fs4 = df[['Phone_new', 'TN', 'Region', 'Area', 'Actions', 'Source']]
tn_values = df['TN'].value_counts().index
for col in tn_values:
    fs4[col] = (fs4['TN'] == col).apply(lambda x: int(x))

actions_values = df['Actions'].value_counts().index
actions_set = set()
for action in actions_values:
    values = action.split(',')
    for value in values:
        actions_set.add(value)

for col in list(actions_set):
    fs4[col] = fs4['Actions'].apply(lambda x: int(str(x).find(col) != -1))

source_values = df['Source'].value_counts().index
for col in source_values:
    fs4[col] = (fs4['Source'] == col).apply(lambda x: int(x))

fs4['Большой город'] = fs4['Region'].apply(lambda x: int(str(x) in ['Москва', 'Санкт-Петербург']))
fs4.drop(columns=['TN', 'Region', 'Area', 'Actions', 'Source'], inplace=True)

fs4_agg = fs4.groupby(['Phone_new']).sum()

fs = fs3.join(fs4_agg)
fs.fillna(0, inplace=True)
fs


/var/folders/bh/d932d5zj56z9_y_7lh3s66ndf1n4c9/T/ipykernel_19883/3099112782.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fs4[col] = (fs4['TN'] == col).apply(lambda x: int(x))
/var/folders/bh/d932d5zj56z9_y_7lh3s66ndf1n4c9/T/ipykernel_19883/3099112782.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fs4[col] = (fs4['TN'] == col).apply(lambda x: int(x))
/var/folders/bh/d932d5zj56z9_y_7lh3s66ndf1n4c9/T/ipykernel_19883/3099112782.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

,Чеки,Товары,Выручка,Товар/чек,Средний_чек,Чеки_выкуп,Товары_выкуп,Выручка_выкуп,Товар/чек_выкуп,Средний_чек_выкуп,...,Подарки,СкидкаДляСпособаОплаты,НайдёшьДешевле,СкидкаНаКомплект,СкидкаСуммойПроцентомСЛимитом,Онлайн-Резерв.,Колл-центр,Сайт,Терминал,Большой город
Phone_new,,,,,,,,,,,,,,,,,,,,,
32565748-000000000,13,15,259458.0,1.15,19958.31,8.0,8.0,19757.0,1.0,2469.62,...,0,0,0,0,1,0,0,0,15,14
32565748-535549565054 55,1,1,12430.0,1.00,12430.00,0.0,0.0,0.0,0.0,0.00,...,0,0,0,0,0,1,0,0,0,1
32565750-575448565151 а,1,2,53198.0,2.00,53198.00,1.0,2.0,53198.0,2.0,53198.00,...,0,0,0,0,0,0,2,0,0,0
32565750-575448565151 а5,1,1,1590.0,1.00,1590.00,1.0,1.0,1590.0,1.0,1590.00,...,0,0,0,0,0,1,0,0,0,1
43553240-575054413251+75,7,7,48557.0,1.00,6936.71,2.0,2.0,10129.0,1.0,5064.50,...,0,0,0,0,0,7,0,0,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57505548-48535350504991,1,1,999.0,1.00,999.00,1.0,1.0,999.0,1.0,999.00,...,0,0,0,0,0,0,0,1,0,1
57544955-51485756555597,1,1,7700.0,1.00,7700.00,0.0,0.0,0.0,0.0,0.00,...,0,0,0,0,1,1,0,0,0,0
57555656-55575253555698,1,1,4040.0,1.00,4040.00,1.0,1.0,4040.0,1.0,4040.00,...,0,0,0,0,0,1,0,0,0,1


In [4]:
fs.reset_index().to_csv('features.csv', index=False)